In [11]:
import os
import torch
from torch.utils.data import Dataset
from skimage import io
from PIL import Image
import numpy as np

class MicroControllerDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

class YoloDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(images_dir) if f.endswith('.jepg')]  # assuming jpg files

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.images_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert('RGB')

        # Load label (YOLO format)
        label_path = os.path.join(self.labels_dir, self.image_filenames[idx].replace('.jepg', '.txt'))
        boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    boxes.append([class_id, x_center, y_center, width, height])

        boxes = np.array(boxes) if boxes else np.zeros((0, 5))

        # Transform image if needed
        if self.transform:
            image = self.transform(image)

        # Return image and target (bounding boxes and labels)
        target = {
            'boxes': torch.tensor(boxes[:, 1:], dtype=torch.float32),  # x_center, y_center, width, height
            'labels': torch.tensor(boxes[:, 0], dtype=torch.long)  # class ids
        }

        return image, target


In [13]:
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((640, 640)),  # Resizing to YOLOv5 size
    T.ToTensor(),
])

# Load dataset
dataset = YoloDataset(images_dir='assets', labels_dir='assets', transform=transform)

In [14]:
# Create DataLoader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
def collate_fn(batch):
    images, targets = list(zip(*batch))
    images = torch.stack(images, dim=0)
    targets = [{k: v for k, v in t.items()} for t in targets]
    return images, targets


In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
